## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/phonocardiogram/data/PV"

In [4]:
csv = pd.read_csv("/Users/ecem/Desktop/phonocardiogram/data/fixed_csv.csv", index_col = 0)

In [5]:
csv.head()

,Patient ID,Age,Sex,Height,Weight,Pregnancy status,Murmur,Most audible location,Systolic murmur timing,Systolic murmur shape,...,Campaign,Additional ID,recording location AV,recording location MV,recording location PV,recording location TV,murmur location AV,murmur location MV,murmur location PV,murmur location TV
0,2530,Child,Female,98.0,15.9,False,Absent,NaN,NaN,NaN,...,CC2015,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,9979,Child,Female,103.0,13.1,False,Present,TV,Holosystolic,Diamond,...,CC2015,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,9983,Child,Male,115.0,19.1,False,Unknown,NaN,NaN,NaN,...,CC2015,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,13918,Child,Male,98.0,15.9,False,Present,TV,Holosystolic,Plateau,...,CC2015,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
4,14241,Child,Male,87.0,11.2,False,Present,PV,Early-systolic,Plateau,...,CC2015,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### get indexes for recordings taken in specific location with murmur = absent, present

In [6]:
#this shows indexes of recordings taken in av location which also has a murmur at av
idx_pv_present =  np.where((csv['Murmur'] == 'Present') & (csv['murmur location PV'] == 1) &
                           (csv['recording location PV'] == 1))
idx_pv_present

(array([  1,   4,   8,  12,  19,  21,  23,  25,  33,  42,  43,  44,  55,
         57,  63,  64,  68,  70,  82,  85, 106, 135, 136, 164, 176, 187,
        203, 216, 219, 220, 231, 233, 242, 244, 248, 269, 272, 332, 358,
        359, 360, 367, 389, 416, 418, 429, 430, 434, 440, 441, 446, 453,
        463, 466, 469, 471, 479, 485, 488, 498, 502, 527, 528, 529, 531,
        534, 539, 551, 555, 572, 573, 588, 594, 609, 628, 634, 650, 659,
        665, 666, 667, 678, 705, 711, 743, 752, 757, 758, 762, 765, 772,
        773, 782, 783, 787, 800, 807, 815, 816, 844, 856, 857, 860, 867,
        881, 892, 896]),)

In [7]:
len(idx_pv_present[0])

107

In [8]:
#this shows indexes of recordings taken in av location which also has not a murmur at anywhere
idx_pv_absent =  np.where((csv['Murmur'] == 'Absent') & (csv['murmur location PV'] == 0) &
                           (csv['recording location PV'] == 1))
idx_pv_absent

(array([  0,   5,   6,   7,  10,  14,  15,  17,  18,  20,  29,  30,  31,
         34,  35,  39,  47,  48,  49,  52,  53,  54,  58,  59,  61,  71,
         74,  75,  77,  78,  80,  81,  84,  87,  89,  90,  95,  98,  99,
        102, 104, 105, 107, 109, 110, 114, 115, 118, 119, 120, 122, 125,
        130, 133, 134, 138, 141, 143, 145, 147, 148, 149, 150, 152, 153,
        155, 158, 159, 161, 162, 163, 173, 175, 191, 196, 199, 204, 205,
        207, 209, 210, 214, 217, 218, 221, 223, 224, 227, 229, 234, 235,
        237, 238, 239, 240, 241, 243, 246, 249, 250, 251, 252, 254, 256,
        258, 260, 261, 263, 264, 267, 268, 270, 274, 278, 281, 284, 285,
        286, 287, 288, 290, 291, 292, 293, 294, 296, 297, 300, 306, 313,
        315, 316, 318, 321, 322, 323, 325, 327, 328, 329, 331, 333, 335,
        337, 338, 339, 340, 341, 342, 343, 346, 347, 348, 350, 352, 353,
        357, 362, 364, 365, 369, 371, 372, 373, 374, 375, 376, 378, 379,
        380, 381, 383, 384, 385, 386, 391, 393, 394

In [9]:
len(idx_pv_absent[0])

509

## now lets get the data in which murmur is present

### murmur = present

In [10]:
ids_pv_present = csv.loc[idx_pv_present]['Patient ID'].values
ids_pv_present

array([ 9979, 14241, 29045, 36327, 40840, 44514, 46065, 46579, 49574,
       49627, 49628, 49630, 49691, 49712, 49751, 49754, 49821, 49824,
       49897, 49931, 49994, 50054, 50056, 50115, 50129, 50150, 50209,
       50229, 50233, 50238, 50256, 50260, 50277, 50280, 50289, 50326,
       50331, 50676, 50734, 50735, 50736, 50744, 50781, 55945, 57706,
       68194, 68204, 68255, 68303, 68306, 68337, 68379, 68425, 68432,
       68444, 68456, 68487, 68560, 68582, 68708, 68740, 69068, 69079,
       69093, 69096, 69120, 69147, 73497, 76240, 84692, 84693, 84718,
       84730, 84751, 84786, 84802, 84834, 84854, 84863, 84864, 84865,
       84882, 84937, 84949, 85002, 85024, 85030, 85031, 85036, 85042,
       85055, 85057, 85079, 85080, 85087, 85110, 85119, 85132, 85133,
       85172, 85198, 85199, 85207, 85219, 85243, 85261, 85269])

In [11]:
len(ids_pv_present)

107

In [12]:
np.save(path + "/murmur_present_ids", ids_pv_present, allow_pickle = True)

In [13]:
ids_pv_absent = csv.loc[idx_pv_absent]['Patient ID'].values
ids_pv_absent

array([ 2530, 14998, 23625, 24160, 31737, 39043, 39403, 40058, 40798,
       43852, 49561, 49562, 49568, 49577, 49585, 49610, 49641, 49653,
       49659, 49678, 49683, 49687, 49719, 49729, 49745, 49829, 49839,
       49842, 49853, 49854, 49876, 49896, 49930, 49952, 49960, 49963,
       49974, 49980, 49983, 49988, 49990, 49993, 49995, 49999, 50001,
       50007, 50008, 50014, 50015, 50017, 50023, 50029, 50043, 50049,
       50053, 50061, 50070, 50074, 50076, 50078, 50079, 50080, 50085,
       50089, 50092, 50096, 50103, 50104, 50109, 50111, 50113, 50126,
       50128, 50155, 50165, 50174, 50210, 50213, 50216, 50218, 50219,
       50225, 50230, 50231, 50239, 50244, 50247, 50250, 50254, 50261,
       50263, 50271, 50272, 50273, 50275, 50276, 50278, 50284, 50291,
       50295, 50296, 50297, 50299, 50303, 50306, 50312, 50314, 50317,
       50318, 50323, 50325, 50327, 50334, 50339, 50343, 50349, 50350,
       50352, 50354, 50359, 50379, 50384, 50385, 50386, 50388, 50393,
       50619, 50624,

In [14]:
len(ids_pv_absent)

509

In [15]:
np.save(path + "/murmur_absent_ids", ids_pv_absent, allow_pickle = True)